In [2]:
!sh scripts/PatchTST/weather_quick.sh 

Args in experiment:
Namespace(activation='gelu', affine=0, batch_size=8, c_out=7, checkpoints='./checkpoints/', d_ff=64, d_layers=1, d_model=32, data='custom', data_path='weather.csv', dec_in=7, decomposition=0, des='Exp', devices='0,1,2,3', distil=True, do_predict=False, dropout=0.1, e_layers=1, embed='timeF', embed_type=0, enc_in=21, factor=1, fc_dropout=0.1, features='M', freq='h', gpu=0, head_dropout=0.0, individual=0, is_training=1, itr=1, kernel_size=25, label_len=48, learning_rate=0.0001, loss='mse', lradj='type3', model='PatchTST', model_id='weather_quick_test_48_24', moving_avg=25, n_heads=4, num_workers=10, output_attention=False, padding_patch='end', patch_len=4, patience=1, pct_start=0.3, pred_len=24, random_seed=2021, revin=1, root_path='./dataset/weather/', seq_len=48, stride=2, subtract_last=0, target='OT', test_flop=False, train_epochs=1, use_amp=False, use_gpu=False, use_multi_gpu=False)
Use CPU
>>>>>>>start training : weather_quick_test_48_24_PatchTST_custom_ftM_sl48_

In [52]:
import torch
import numpy as np
import pandas as pd
from data_provider.data_loader import Dataset_Custom
from models.PatchTST import Model

# Define model parameters to match training
class Configs:
    enc_in = 21
    seq_len = 48
    pred_len = 24
    e_layers = 1
    n_heads = 4
    d_model = 32
    d_ff = 64
    dropout = 0.1
    fc_dropout = 0.1
    head_dropout = 0
    patch_len = 4  # Match with training configuration
    stride = 4
    individual = False
    revin = True
    affine = False
    subtract_last = False
    decomposition = False
    kernel_size = 25
    padding_patch = 'end'
    max_seq_len = 1024
    d_k = None
    d_v = None
    norm = 'BatchNorm'
    attn_dropout = 0.0
    act = 'gelu'
    key_padding_mask = 'auto'
    padding_var = None
    attn_mask = None
    res_attention = True
    pre_norm = False
    store_attn = False
    pe = 'zeros'
    learn_pe = True
    pretrain_head = False
    head_type = 'flatten'
    verbose = False

configs = Configs()

# Load model checkpoint
checkpoint_path = '/home/jaleed/PatchTST/PatchTST_supervised/checkpoints/weather_quick_test_48_24_PatchTST_custom_ftM_sl48_ll48_pl24_dm32_nh4_el1_dl1_df64_fc1_ebtimeF_dtTrue_Exp_0/checkpoint.pth'
model = Model(configs)

# Load the state dictionary
state_dict = torch.load(checkpoint_path)

# Debugging statement: Print keys of the state_dict
print("Keys in the loaded state_dict:")
print(state_dict.keys())

# Debugging statement: Print shapes of tensors in the state_dict
for k, v in state_dict.items():
    print(f"{k}: {v.shape}")

# Attempt to load the state dictionary into the model
try:
    model.load_state_dict(state_dict)
    print("Model state dictionary loaded successfully.")
except RuntimeError as e:
    print(f"RuntimeError during model state_dict loading: {e}")

model.eval()

# Load the test dataset
test_data_path = '/home/jaleed/PatchTST/PatchTST_supervised/dataset/weather/weather.csv'
weather_df = pd.read_csv(test_data_path)

# Define the test dataset
test_dataset = Dataset_Custom(root_path='/home/jaleed/PatchTST/PatchTST_supervised/dataset/weather/', data_path='weather.csv', flag='test', size=[configs.seq_len, configs.pred_len, configs.seq_len], features='M', target='OT')

# Get input sequence
input_sequences = []
input_dates = []

# Using a smaller sample size for quick testing
num_samples = 1
for i in range(num_samples):
    sample = test_dataset[i]
    input_sequences.append(sample[0])  # Directly append the sample as it is already a NumPy array
    start_idx = weather_df.shape[0] - len(test_dataset) + i
    input_dates.append(weather_df.iloc[start_idx:start_idx + configs.seq_len]['date'].values)

input_sequences = np.stack(input_sequences)  # Shape should be (num_samples, seq_len, enc_in)
input_tensors = torch.tensor(input_sequences).float()  # Convert to tensor

# Print the input sequence
flattened_input_sequences = input_sequences.reshape(-1, input_sequences.shape[-1])
flattened_dates = np.array(input_dates).reshape(-1)
df_input = pd.DataFrame(flattened_input_sequences, columns=weather_df.columns[1:])
df_input.insert(0, 'date', flattened_dates)
print("Input sequence:")
print(df_input)
print("\n")

# Check the shape of the input tensor
print(f"Shape of input tensor before permutation: {input_tensors.shape}")

# Run the model on the sequence
with torch.no_grad():
    input_tensors = input_tensors.permute(0, 2, 1)  # permute to [batch_size, num_features, seq_len]
    print(f"Shape of input tensor after permutation: {input_tensors.shape}")
    try:
        prediction = model(input_tensors)
    except RuntimeError as e:
        print(f"RuntimeError during model prediction: {e}")

# Convert prediction to DataFrame if successful
if 'prediction' in locals():
    prediction_2d = prediction.squeeze(0).permute(1, 0).numpy()  # Remove the batch dimension and permute back

    # Create DataFrame for prediction
    columns = weather_df.columns[1:]  # Exclude 'date' column
    df_prediction = pd.DataFrame(prediction_2d, columns=columns)

    # Generate timestamps for prediction
    last_timestamp = pd.to_datetime(flattened_dates[-1])
    pred_timestamps = pd.date_range(last_timestamp, periods=configs.pred_len + 1, freq='10T')[1:]  # Generate next 24 timestamps with 10-minute intervals

    df_prediction.insert(0, 'date', pred_timestamps)

    # Print the prediction
    print("Prediction sequence:")
    print(df_prediction)
    print("\n")
else:
    print("Prediction failed due to RuntimeError.")


Keys in the loaded state_dict:
odict_keys(['model.backbone.W_pos', 'model.backbone.W_P.weight', 'model.backbone.W_P.bias', 'model.backbone.encoder.layers.0.self_attn.W_Q.weight', 'model.backbone.encoder.layers.0.self_attn.W_Q.bias', 'model.backbone.encoder.layers.0.self_attn.W_K.weight', 'model.backbone.encoder.layers.0.self_attn.W_K.bias', 'model.backbone.encoder.layers.0.self_attn.W_V.weight', 'model.backbone.encoder.layers.0.self_attn.W_V.bias', 'model.backbone.encoder.layers.0.self_attn.sdp_attn.scale', 'model.backbone.encoder.layers.0.self_attn.to_out.0.weight', 'model.backbone.encoder.layers.0.self_attn.to_out.0.bias', 'model.backbone.encoder.layers.0.norm_attn.1.weight', 'model.backbone.encoder.layers.0.norm_attn.1.bias', 'model.backbone.encoder.layers.0.norm_attn.1.running_mean', 'model.backbone.encoder.layers.0.norm_attn.1.running_var', 'model.backbone.encoder.layers.0.norm_attn.1.num_batches_tracked', 'model.backbone.encoder.layers.0.ff.0.weight', 'model.backbone.encoder.laye

ValueError: cannot select an axis to squeeze out which has size not equal to one